In [1]:
%pip install langchain-core langgraph>0.2.27

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import getpass
os.environ["LANGSMITH_TRACING"]="true"
os.environ["LANGSMITH_API_KEY"]=getpass.getpass()

 ········


In [3]:
%pip install -qU "langchain[groq]"


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
if not os.environ.get("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"]=getpass.getpass("enter groq api key:")

from langchain.chat_models import init_chat_model
model=init_chat_model("deepseek-r1-distill-llama-70b",model_provider="groq")

enter groq api key: ········


In [5]:
from langchain_core.messages import HumanMessage,AIMessage


In [6]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer =trim_messages(
    max_tokens=65,
    strategy="last",
    start_on="human",
    token_counter=model,
    allow_partial=True,
    include_system=True,
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

C:\Users\atul.yadav\langchain-academy\lc-academy-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [7]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt_temp=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "you are like a google , answer all question to best of your knowledge in short and concise manner in {language}."
        ),
        MessagesPlaceholder(variable_name="messages")

    ]
)

In [8]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START,MessagesState,StateGraph
from typing import Sequence 
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

class State(TypedDict):
    messages:Annotated[Sequence[BaseMessage],add_messages]
    language:str
    
workflow =StateGraph(state_schema=State)

async def call_model(state:State):
    trimmed_message=trimmer.invoke(state["messages"])
    
    prompt=prompt_temp.invoke({"messages":trimmed_message,"language":state["language"]})
    response=await model.ainvoke(prompt)
    return {"messages":[response]+trimmed_message}

workflow.add_node("model",call_model)
workflow.add_edge(START,"model")

memory=MemorySaver()
chatbot=workflow.compile(checkpointer=memory)

In [10]:
query="which match problem we talked about"
language="english"
input_message=messages+[HumanMessage(query)]
response = await chatbot.ainvoke({"messages":input_message,"language":language},config={"configurable":{"thread_id":"7"}})
response["messages"][-1].pretty_print()


================================== Ai Message ==================================

<think>
Alright, the user is asking, "which match problem we talked about." Looking back at our conversation, I see that earlier they asked "whats 2 + 2" and I replied with "4." So, they're referring to that math problem.

They might be checking if I remember our previous interaction or perhaps they want to discuss more about it. I should acknowledge the specific problem we solved and offer further assistance in case they have more questions or need help with something else.

I'll phrase my response to confirm that it was the 2 + 2 problem and let them know I'm here to help with anything else they need.
</think>

We talked about solving the math problem **2 + 2**, which equals **4**. Let me know if you need help with anything else!


In [16]:
query="which match problem we talked about"
language="english"
input_message=[HumanMessage(query)]
#response = await chatbot.ainvoke({"messages":input_message,"language":language},config={"configurable":{"thread_id":"8"}})


async for chunk, metadata in chatbot.astream(
    {"messages":input_message,"language":language},config={"configurable":{"thread_id":"8"}},stream_mode="messages",
):
    if isinstance(chunk,AIMessage):
        print(chunk.content,end="|")



|<think>|
 previous| conversation|.| I| need| to| figure| out| what| they|'re| referring| to|. I| remember| they| mentioned| something| about| a| match| problem| before|,| but| I| don|'t| have| the| details| of| our|

 you| have| to| arrange| matches| to| form| certain| shapes| or| numbers|.| There| are| several| classic| problems|,| such| as| forming| a| square| or| a| triangle| with| a| certain| number| of| matches|.

 the| minimum| number| of| matches| needed| to| balance| an| equation| or| form| a| particular| shape|.| Another| possibility| is| that| they|'re| talking| about| a| problem| involving| burning| matches|,| like| the| classic| river| crossing| puzzle| or| something| related| to| time| measurement| with| burning| matches|.

 context| from| our| previous| conversation|,| I| should| ask| for| clarification|.| I|'ll| let| them| know| I| don|'t| recall| the| specifics| and| ask| them| to| provide| more| details| so| I| can| assist| them| better|.| That| way|,| I| can| give| a

In [72]:
query="What is my name"
language="Chinese"
input_message=[HumanMessage(query)]
response = await chatbot.ainvoke({"messages":input_message,"language":language},config={"configurable":{"thread_id":"4"}})
response["messages"][-1].pretty_print()


================================== Ai Message ==================================

<think>
Okay, the user is asking me to act like Google and answer their questions concisely in Chinese. They're starting with "What is my name." Hmm, I don't have access to personal information unless they provide it. I should respond politely that I can't tell them their name because I don't have that data. I need to make sure my answer is in Chinese, short, and clear. Maybe something like, "我无法知道您的名字，因为我无法访问个人信息。"
</think>

我无法知道您的名字，因为我无法访问个人信息。


In [68]:
query="I m Chetan"
language="Spanish"
input_message=[HumanMessage(query)]
response = await chatbot.ainvoke({"messages":input_message,"language":language},config={"configurable":{"thread_id":"3"}})
response["messages"][-1].pretty_print()


================================== Ai Message ==================================

<think>
Okay, the user is asking me to act like Google, answering questions concisely in Spanish. They introduced themselves as Chetan. I should acknowledge that and offer assistance in Spanish, keeping it friendly and open-ended.
</think>

¡Hola Chetan! ¿En qué puedo ayudarte hoy?


In [69]:
#NOW ENTRIE STATE IS PERSISTATE SO WE OMITTED THE LANGUAGE
query="WHAT IS MY NAME"
input_message=[HumanMessage(query)]
response = await chatbot.ainvoke({"messages":input_message},config={"configurable":{"thread_id":"3"}})
response["messages"][-1].pretty_print()


================================== Ai Message ==================================

<think>
Alright, the user has just asked, "WHAT IS MY NAME." Looking back at the history, they introduced themselves as Chetan. So, I should respond with their name. Since they want answers in Spanish, I'll say "Tu nombre es Chetan." That's straightforward and meets their request.
</think>

Tu nombre es Chetan.
